**For analytics part, please refer to: https://colab.research.google.com/drive/16WY1lY-BV66OUyDukSooWs8pi8aQF1ah?usp=sharing**

**For the results file, please refer to: https://drive.google.com/file/d/1dO3gJTSdSQx2Lc0nhi-2e41904udyhq-/view?usp=sharing**

# **Importing libraries and mounting drive**

In [241]:
import pandas as pd
import numpy as np
#import Textdistance
import re
from collections import Counter
import os
!pip install pmdarima
import pmdarima as pm

from google.colab import drive
drive.mount("/content/drive")

#If source code is copied, below drive should be changed to relevant address, where the 'train.csv' file is located and where the results1.csv will be exported

path = '/content/drive/MyDrive/Colab Notebooks/Datathon'

os.chdir(path)
!ls


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Datathon.ipynb	results1.csv  train.csv


# **Importing some more libraries and reading training data**

In [ ]:
from dateutil.parser import parse 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
plt.rcParams.update({'figure.figsize': (10, 7), 'figure.dpi': 120})

df = pd.read_csv('train.csv', parse_dates=['OPERATIONDATE'])


# **Defining ATM's number and number of prediction days**

In [ ]:
atm_list= df.CASHPOINTID.unique()
n_periods=30
atm_preds = []

# **For each ATM finding best ARIMA parameters, fiting the model and predicting**

In [ ]:
for i in range(0, len(atm_list)):
  atm = df.loc[df['CASHPOINTID'] == i]
  atm['OPERATIONDATE'] = pd.to_datetime(atm['OPERATIONDATE'])
  atm = atm.sort_values(by=['OPERATIONDATE'])
  atm = atm.groupby('OPERATIONDATE')['AMT_SCALED'].sum().reset_index()
  atm = atm.set_index('OPERATIONDATE')
  model = pm.auto_arima(atm.AMT_SCALED, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
  model_fit = model.fit(atm) 
  # Forecast
  fc, confint = model_fit.predict(n_periods=n_periods, return_conf_int=True)
  index_of_fc = np.arange(len(atm.AMT_SCALED), len(atm.AMT_SCALED)+n_periods)
  atm_preds.append(fc)
  print("Predictions Number:", i, "Preds:", atm_preds[i])

# **Exporting to CSV**

In [ ]:
results = pd.DataFrame(atm_preds)

In [ ]:
results.to_csv('results1.csv')